In [ ]:
from src.models.formulas import *
import pandas as pd

out_calc = widgets.Output()

def show_device_list(Source):
    _devices_select.options = [s for s in list(records.readings[_test.value]['devices'].keys())]

def commonChannels(selected):
    global commonChannelsList
    commonChannelsList = []
    if (len(selected) == 1):
        commonChannelsList = records.readings[_test.value]['devices'][selected[0]]['data'].columns
    if (len(selected) > 1):
        commonChannelsList = records.readings[_test.value]['devices'][selected[0]]['data'].columns
        for s in list(selected):
            commonChannelsList = list(set(commonChannelsList) & set(records.readings[_test.value]['devices'][s]['data'].columns))
    _Aterm.options = list(commonChannelsList)
    _Aterm.source = selected
    _Bterm.options = list(commonChannelsList)
    _Bterm.source = selected
    _Cterm.options = list(commonChannelsList)
    _Cterm.source = selected
    _Dterm.options = list(commonChannelsList)
    _Dterm.source = selected
    
def calculateFormula(b):
    with out_calc:
        clear_output()
        terms = [_Aterm.value, _Bterm.value, _Cterm.value, _Dterm.value]
        for s in list(selected):
            records.addChannelFormula(_test.value, s, _formulaName.value, terms, _formula.value)
            print ("Formula {} Added in test {}, device {}".format(_formulaName.value, _test.value, s))

selected=tuple()
def selectedDevices(Source):
    global selected
    selected = list(Source)
    commonChannels(selected)

# Test dropdown
layout = widgets.Layout(width='350px')
_test = widgets.Dropdown(options=[k for k in records.readings.keys()], 
                        layout=layout,
                        description = 'Test')

_Aterm = widgets.Dropdown(description = 'A', layout=layout)
_Bterm = widgets.Dropdown(description = 'B', layout=layout)
_Cterm = widgets.Dropdown(description = 'C', layout=layout)
_Dterm = widgets.Dropdown(description = 'D', layout=layout)
_devices_select = widgets.SelectMultiple(options = records.readings[_test.value]['devices'].keys(),
                                         layout=widgets.Layout(width='700px'))

_test_drop = widgets.interactive(show_device_list, 
                                Source=_test, 
                                layout=layout)

_selectBox = widgets.VBox([_test_drop])

_formulaName = widgets.Text(description = 'Name: ')
_formula = widgets.Text(description = '=')
_ABtermsBox = widgets.HBox([_Aterm, _Bterm])
_CDtermsBox = widgets.HBox([_Cterm, _Dterm])
_termsBox = widgets.VBox([_ABtermsBox, _CDtermsBox])
_calculate = widgets.Button(description='Calculate')
_calculateBox = widgets.HBox([_formulaName,_formula, _calculate])
_calculate.on_click(calculateFormula)

display(widgets.HTML('<h3>Calculator</h3>'))
display(widgets.HTML('<h4>Select the Files for your formulas to apply</h4>'))
display(_selectBox)

_devices_select_drop = interact(selectedDevices,
                                 Source = _devices_select)

display(widgets.HTML('<h4>Select terms</h4>'))
display(_termsBox)
display(widgets.HTML('<h4>Input your formula Below</h4>'))
display(_calculateBox)

display(out_calc)

## Vapour equilibrium: B is temperature in degC, assumed 1013mbar
# (1.0007 + 3.46*1e-6*1013)*6.1121*np.exp(17.502*B/(240.97+B))